In [ ]:
import random
import torch
from d2l import torch as d2l

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
def synthetic_data(w, b, num_examples):
    # 生成y=wx+b+噪音
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape) # 加个噪音
    return X, y.reshape((-1, 1))

w = torch.tensor([2, -3.4])
b = torch.tensor(4.2)
features, labels = synthetic_data(w, b, 1000)
print(features)
print(labels)

In [ ]:
d2l.plt.scatter(features[:, 0].cpu().numpy(), labels.cpu().numpy(), 1)

In [ ]:
# 每次读取一个mini-batch的数据来训练
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices) # 打乱下标
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i : min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices] # 生成了一个iterator,它会记住上次调用的位置。下次继续调用时会回到上次调用的结果后面


batch_size = 10
# for X, y in data_iter(batch_size, features, labels):
#     print(X, '\n', y)

In [ ]:
def linreg(X, w, b):
    return torch.matmul(X, w) + b # 这是我们的模型，产生预测值的公式

w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad = True)

# 均方损失函数
def loss(w, b, X, y):
    return torch.mean(torch.square(linreg(X, w, b)- y)) / 2 # torch.mean()返回整个矩阵元素的均值

# 更新参数
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad # param.grad是torch自动算出来的
            param.grad.zero_()

In [ ]:
lr = 0.03
num_epoch = 10
net = linreg
loss = loss

for epoch in range(num_epoch):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(w, b, X, y) # 每次读一小批数据出来算loss，然后反向传播一下
        l.backward() # 之前已经把loss整成了标量，所以这里不需要sum了
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(w, b, features, labels)
        print('epoch @ {}: loss {}'.format(epoch + 1, float(train_l)))

print(w)
print(b)